# camera_geometry

This notebook implements core camera geometry routines used in robotics and vision:

- Build intrinsic camera matrix (K)
- Project 3D points to 2D image plane
- Optional image distortion (radial + tangential)
- Undistort points / images
- Backproject depth map to 3D points


## Camera intrinsics and projection (theory)

A pinhole camera projects 3D points `(X, Y, Z)` (in camera coordinates) to 2D pixels `(u, v)` via:

1. Convert to normalized image coordinates:
   $$
   x = X/Z,\quad y = Y/Z
   $$

2. Apply camera intrinsics (focal lengths and principal point):
   $$
   u = f_x x + c_x,\quad v = f_y y + c_y
   $$

Matrix form using homogeneous coordinates:
$$
\begin{bmatrix} u \\ v \\ 1 \end{bmatrix}
\sim
\begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix} X/Z \\ Y/Z \\ 1 \end{bmatrix}
$$

Distortion (OpenCV radial+tangential model):
$$
r^2 = x^2 + y^2
$$
$$
x_{d} = x(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) + 2 p_1 x y + p_2 (r^2 + 2 x^2)
$$
$$
y_{d} = y(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) + p_1 (r^2 + 2 y^2) + 2 p_2 x y
$$

To undistort we solve for `(x, y)` given `(x_d, y_d)` — commonly done iteratively (OpenCV uses its own solvers).


In [1]:
# Imports & helpers
import numpy as np
import math
from typing import Tuple, Optional
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage

# Utility to ensure arrays shape
def _to_np(x):
    return np.asarray(x, dtype=np.float64)


In [2]:
def build_intrinsic(fx: float, fy: Optional[float] = None,
                    cx: float = 0.0, cy: float = 0.0,
                    skew: float = 0.0) -> np.ndarray:
    """
    Build camera intrinsic matrix K.
      fx, fy: focal lengths in pixels (fy defaults to fx if None)
      cx, cy: principal point (pixel coordinates)
      skew : non-zero if pixels are skewed (rare)
    Returns:
      K (3x3)
    """
    fx = float(fx)
    fy = fx if fy is None else float(fy)
    K = np.array([[fx, skew, cx],
                  [0.0, fy, cy],
                  [0.0, 0.0, 1.0]], dtype=np.float64)
    return K

# Example
K_example = build_intrinsic(500, None, 320, 240)
print("K_example:\n", K_example)


K_example:
 [[500.   0. 320.]
 [  0. 500. 240.]
 [  0.   0.   1.]]


In [3]:
def project_points(points_3d: np.ndarray, K: np.ndarray) -> np.ndarray:
    """
    Project Nx3 points in camera coordinates to Nx2 pixel coordinates (no distortion).
    points_3d: (N,3) array
    K: (3,3) intrinsic matrix
    Returns:
      px: (N,2) pixel coordinates (u,v)
      z:  (N,) depth values (Z)
    """
    pts = _to_np(points_3d)
    assert pts.ndim == 2 and pts.shape[1] == 3, "points_3d must be (N,3)"
    X = pts[:,0]; Y = pts[:,1]; Z = pts[:,2]
    eps = 1e-9
    Z_safe = np.where(Z == 0, eps, Z)
    x = X / Z_safe
    y = Y / Z_safe
    # apply intrinsics
    u = K[0,0] * x + K[0,2]
    v = K[1,1] * y + K[1,2]
    return np.stack([u, v], axis=1), Z

# Small test
pts = np.array([[0.1, 0.0, 1.0], [0.0, 0.1, 1.0], [0.0,0.0,2.0]])
px, depth = project_points(pts, K_example)
print("pixels:\n", px)
print("depths:\n", depth)


pixels:
 [[370. 240.]
 [320. 290.]
 [320. 240.]]
depths:
 [1. 1. 2.]


In [4]:
def distort_points(xy: np.ndarray,
                   k: Tuple[float,float,float] = (0.0,0.0,0.0),
                   p: Tuple[float,float] = (0.0,0.0)) -> np.ndarray:
    """
    Apply OpenCV-style distortion to normalized coordinates.
      xy: (N,2) array in normalized coords x = X/Z, y = Y/Z
      k = (k1,k2,k3) radial params
      p = (p1,p2) tangential params
    Returns:
      (N,2) distorted normalized coordinates (x_d, y_d)
    """
    xy = _to_np(xy)
    x = xy[:,0]; y = xy[:,1]
    k1, k2, k3 = k
    p1, p2 = p
    r2 = x*x + y*y
    radial = 1 + k1*r2 + k2*(r2**2) + k3*(r2**3)
    x_d = x * radial + 2*p1*x*y + p2*(r2 + 2*x*x)
    y_d = y * radial + p1*(r2 + 2*y*y) + 2*p2*x*y
    return np.stack([x_d, y_d], axis=1)

# Quick check
xy = np.array([[0.1, 0.1], [0.0, 0.5]])
print("distorted:", distort_points(xy, k=(0.1,-0.05,0.0), p=(0.001, -0.002)))


distorted: [[ 1.001380e-01  1.001980e-01]
 [-5.000000e-04  5.116875e-01]]


In [5]:
def project_points_distorted(points_3d: np.ndarray, K: np.ndarray,
                             k=(0.0,0.0,0.0), p=(0.0,0.0)) -> Tuple[np.ndarray, np.ndarray]:
    """
    Project 3D points with distortion applied.
    Returns pixel coords (N,2) and depth (N,)
    """
    pts = _to_np(points_3d)
    X = pts[:,0]; Y = pts[:,1]; Z = pts[:,2]
    Z_safe = np.where(Z == 0, 1e-9, Z)
    xy = np.stack([X / Z_safe, Y / Z_safe], axis=1)
    xy_d = distort_points(xy, k=k, p=p)
    u = K[0,0] * xy_d[:,0] + K[0,2]
    v = K[1,1] * xy_d[:,1] + K[1,2]
    return np.stack([u, v], axis=1), Z

# Quick demo
pts = np.array([[0.2,0.1,1.0], [0.0,0.5,1.0]])
px_d, z = project_points_distorted(pts, K_example, k=(0.1, -0.05, 0.0), p=(0.001,0.0))
print("distorted pixels:", px_d)


distorted pixels: [[420.5075  290.27875]
 [320.      495.84375]]


In [6]:
def undistort_points_normalized(xy_d: np.ndarray, k=(0.0,0.0,0.0), p=(0.0,0.0),
                                max_iter: int = 20, tol: float = 1e-9) -> np.ndarray:
    """
    Given distorted normalized coords xy_d, find the undistorted normalized coords xy.
    Uses iterative inverse solving (Newton-like fixed point).
    xy_d: (N,2)
    Returns: xy (N,2)
    """
    xy_d = _to_np(xy_d)
    x_u = xy_d.copy()  # initial guess: assume distortion small
    for it in range(max_iter):
        x = x_u[:,0]; y = x_u[:,1]
        r2 = x*x + y*y
        k1, k2, k3 = k
        p1, p2 = p
        radial = 1 + k1*r2 + k2*(r2**2) + k3*(r2**3)
        x_calc = x * radial + 2*p1*x*y + p2*(r2 + 2*x*x)
        y_calc = y * radial + p1*(r2 + 2*y*y) + 2*p2*x*y
        err = np.stack([x_calc, y_calc], axis=1) - xy_d
        x_u = x_u - err  # simple fixed-point correction (open-loop)
        if np.max(np.abs(err)) < tol:
            break
    return x_u

# simple unit check: distort -> undistort
xy = np.array([[0.12, 0.08], [0.2, -0.1]])
k = (0.1, -0.05, 0.0); p = (0.001, 0.0005)
xy_d = distort_points(xy, k=k, p=p)
xy_hat = undistort_points_normalized(xy_d, k=k, p=p)
print("orig:", xy)
print("recovered:", xy_hat)
print("error:", np.linalg.norm(xy - xy_hat, axis=1))


orig: [[ 0.12  0.08]
 [ 0.2  -0.1 ]]
recovered: [[ 0.12  0.08]
 [ 0.2  -0.1 ]]
error: [5.87712435e-15 6.79210912e-13]


In [7]:
def undistort_image(img: np.ndarray, K: np.ndarray,
                    dist_coeffs: Tuple[float,float,float,float,float] = (0,0,0,0,0)) -> np.ndarray:
    """
    Undistort image using OpenCV if available. dist_coeffs = (k1,k2,p1,p2,k3)
    Fallback: naive per-pixel inverse mapping using iterative undistort solver + bilinear interp (slower).
    """

    h,w = img.shape[:2]
    # OpenCV expects dist = [k1,k2,p1,p2,k3]
    dist = np.array(dist_coeffs, dtype=np.float64)
    # new camera matrix = same as K (we don't optimize here)
    newK = K.copy()
    # undistort
    und = cv2.undistort(img, K.astype(np.float64), dist, newCameraMatrix=newK)
    return und



In [8]:
def backproject_depth(depth: np.ndarray, K: np.ndarray) -> np.ndarray:
    """
    Convert a depth map (H,W) to an (H*W, 3) array of 3D points in camera coordinates.
    depth: (H,W) depth in same units as camera (Z)
    K: intrinsic matrix
    Returns:
      points_3d: (H*W, 3)
    """
    depth = np.asarray(depth, dtype=np.float64)
    H, W = depth.shape
    i_coords, j_coords = np.indices((H, W))
    u = j_coords.ravel().astype(np.float64)
    v = i_coords.ravel().astype(np.float64)
    fx, fy = K[0,0], K[1,1]
    cx, cy = K[0,2], K[1,2]
    Z = depth.ravel()
    # ignore zero depth (set small eps to avoid divide)
    valid = Z > 0
    X = ((u - cx) / fx) * Z
    Y = ((v - cy) / fy) * Z
    pts = np.stack([X, Y, Z], axis=1)
    return pts, valid

# test: produce a depth map where all depths = 2.0 and backproject single pixel
depth = np.ones((2,3))*2.0
pts3d, valid = backproject_depth(depth, K_example)
print("points_3d shape:", pts3d.shape)
print("first points:\n", pts3d[:6])


points_3d shape: (6, 3)
first points:
 [[-1.28  -0.96   2.   ]
 [-1.276 -0.96   2.   ]
 [-1.272 -0.96   2.   ]
 [-1.28  -0.956  2.   ]
 [-1.276 -0.956  2.   ]
 [-1.272 -0.956  2.   ]]


In [9]:
# Demo: cube points, project, distort, undistort -> backproject error
def demo_cube_reconstruction(K, k=(0.0,0.0,0.0), p=(0.0,0.0)):
    # create cube corners in camera coords (units: meters)
    cube = np.array([[x,y,z] for x in (-0.3,0.3) for y in (-0.3,0.3) for z in (1.0,1.5)])
    # project with distortion
    px_d, z = project_points_distorted(cube, K, k=(k[0], k[1], k[2]), p=p)
    # undistort pixels to normalized coords
    # convert pixels to normalized distorted coords
    xy_d = np.stack([(px_d[:,0] - K[0,2]) / K[0,0], (px_d[:,1] - K[1,2]) / K[1,1]], axis=1)
    xy = undistort_points_normalized(xy_d, k=(k[0],k[1],k[2]), p=p)
    # backproject using recovered normalized coords and depth
    X_rec = np.stack([xy[:,0]*z, xy[:,1]*z, z], axis=1)
    # compute reconstruction error
    err = np.linalg.norm(cube - X_rec, axis=1)
    return cube, px_d, X_rec, err

K = build_intrinsic(800, None, 320, 240)
k = (0.05, -0.02, 0.001); p=(1e-4, -2e-4)
cube, px_d, X_rec, err = demo_cube_reconstruction(K, k=k, p=p)
print("Reconstruction RMS error:", np.sqrt((err**2).mean()))
print("original[0], reconstructed[0]:", cube[0], X_rec[0])


Reconstruction RMS error: 1.950100683988133e-11
original[0], reconstructed[0]: [-0.3 -0.3  1. ] [-0.3 -0.3  1. ]


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def interactive_proj(fx, cx, cy, k1, k2, p1, p2):
    K = build_intrinsic(fx, None, cx, cy)
    k = (k1, k2, 0.0)
    p = (p1, p2)
    # sample random hemisphere points in front of camera
    rng = np.random.RandomState(0)
    N = 400
    X = rng.uniform(-0.8, 0.8, size=(N,))
    Y = rng.uniform(-0.8, 0.8, size=(N,))
    Z = rng.uniform(0.8, 2.0, size=(N,))
    pts = np.stack([X,Y,Z], axis=1)
    px_nd, _ = project_points_distorted(pts, K, k=k, p=p)
    plt.figure(figsize=(6,4))
    plt.scatter(px_nd[:,0], px_nd[:,1], s=6, alpha=0.8)
    plt.gca().invert_yaxis()
    plt.title(f"Projected pixels (fx={fx:.1f}, cx={cx:.1f}, cy={cy:.1f})")
    plt.xlim(0, 640); plt.ylim(0, 480)
    plt.show()

w = widgets.interactive(interactive_proj,
                        fx=widgets.FloatSlider(value=800, min=200, max=2000, step=10),
                        cx=widgets.FloatSlider(value=320, min=0, max=640, step=1),
                        cy=widgets.FloatSlider(value=240, min=0, max=480, step=1),
                        k1=widgets.FloatSlider(value=0.0, min=-0.3, max=0.3, step=0.01),
                        k2=widgets.FloatSlider(value=0.0, min=-0.1, max=0.1, step=0.005),
                        p1=widgets.FloatSlider(value=0.0, min=-0.001, max=0.001, step=1e-5),
                        p2=widgets.FloatSlider(value=0.0, min=-0.001, max=0.001, step=1e-5))
display(w)


interactive(children=(FloatSlider(value=800.0, description='fx', max=2000.0, min=200.0, step=10.0), FloatSlide…